In [1]:
import os
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
path = "Z:\data\BTCUSDT\BTCUSDT-1h"

In [3]:
filelist = os.listdir(path)
filelist.sort()
df = pd.DataFrame()
for file in tqdm(filelist[-400:]):
    df0 = pd.read_csv(os.path.join(path, file))
    df = pd.concat([df, df0], ignore_index=True)

100%|██████████| 400/400 [00:03<00:00, 116.31it/s]


In [13]:
df1 = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
df1.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df1 = df1.dropna()

In [14]:
# 计算ATR平均真实波动
def ATR(df, length=14):
    df = df.copy()
    # 计算真实波动(TR)
    df['H-L'] = df['h'] - df['l']
    df['H-PreClose'] = np.abs(df['h'] - df['c'].shift(1))
    df['L-PreClose'] = np.abs(df['l'] - df['c'].shift(1))
    df['TR'] = df[['H-L', 'H-PreClose', 'L-PreClose']].max(axis=1)

    # 计算ATR
    df['ATR'] = 0
    df = df.reset_index(drop=True)  # 在关键步骤重置索引
    for i in range(length - 1, len(df)):
        if i == length - 1:
            df.loc[df.index[i], 'ATR'] = np.mean((df.loc[df.index[0:i + 1], 'TR'])) / length
        else:
            df.loc[df.index[i], 'ATR'] = (df.loc[df.index[i - 1], 'ATR'] * (length - 1) + df.loc[
                df.index[i], 'TR']) / length
    return df['ATR']

In [15]:
length = 14
df1['g1.ATR'] = 0
df1['g1.ATR'] = ATR(df=df1, length=length)
df1 = df1[length - 1:]  # 去除空值
df1

C:\Users\yyx\AppData\Local\Temp\ipykernel_20092\507226766.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.554591836734679' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df.index[i], 'ATR'] = np.mean((df.loc[df.index[0:i + 1], 'TR'])) / length


,t,o,h,l,c,v,g1.ATR
13,1662382800000,19715.9,19782.0,19702.0,19731.0,16609.655,7.554592
14,1662386400000,19731.0,19889.1,19713.7,19834.6,22538.848,19.543550
15,1662390000000,19834.7,19944.0,19827.5,19848.1,48909.574,26.469010
16,1662393600000,19848.1,19904.5,19815.0,19839.4,18902.583,30.971224
17,1662397200000,19839.4,19850.8,19780.7,19816.9,10108.872,33.766136
...,...,...,...,...,...,...,...
9595,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886,142.023827
9596,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344,139.100697
9597,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588,135.229218
9598,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446,130.212845


In [16]:
df1.to_csv("ATRH.csv",index=False)